In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
nomask_dir = './data/no_mask/'
mask_dir = './data/mask/'

nomask_li = [os.path.join(nomask_dir, file) for file in os.listdir(nomask_dir)[:6900]] # path에 존재하는 파일 목록 가져오기
mask_li = [os.path.join(mask_dir, file) for file in os.listdir(mask_dir)[:6900]] # path에 존재하는 파일 목록 가져오기

nomask_num = len(nomask_li)
mask_num = len(mask_li)

file_num = nomask_num + mask_num

In [3]:
from tqdm import tqdm 

#%% 이미지 전처리
num = 0
all_img = np.float32(np.zeros((file_num, 224, 224, 3))) 
all_label = np.float64(np.zeros((file_num, 1)))

for img_path in tqdm(nomask_li, desc='nomask'):
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 0 # nomask
    num = num + 1
    
for img_name in tqdm(mask_li, desc='mask'):
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 1 # mask
    num = num + 1

mask: 100%|██████████| 6900/6900 [00:13<00:00, 497.14it/s]


In [4]:
# 데이터셋 섞기(적절하게 훈련되게 하기 위함) 
n_elem = all_label.shape[0]
indices = np.random.choice(n_elem, size=n_elem, replace=False)

all_label = all_label[indices]
all_img = all_img[indices]

In [5]:
# 훈련셋 테스트셋 분할
num_train = int(np.round(all_label.shape[0]*0.8))
num_test = int(np.round(all_label.shape[0]*0.2))

train_img = all_img[0:num_train, :, :, :]
test_img = all_img[num_train:, :, :, :] 

train_label = all_label[0:num_train]
test_label = all_label[num_train:]

In [7]:
# create the base pre-trained model
IMG_SHAPE = (224, 224, 3)

base_model = ResNet50(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
base_model.trainable = False
base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))

flatten_layer = Flatten()
dense_layer1 = Dense(128, activation='relu')
bn_layer1 = BatchNormalization()
dense_layer2 = Dense(1, activation=tf.nn.sigmoid)

model = Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        bn_layer1,
        dense_layer2,
        ])

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               12845184  
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 36433537 (138.98 MB)
Trainable params: 12845569 (49.00 MB)
Non-trainable params: 23587968 (89.98 MB)
_____

In [ ]:
model.fit(train_img, train_label, epochs=10, batch_size=16, validation_data = (test_img, test_label))

In [ ]:
# save model
model.save("saved_model_mayhun.h5")